Сначала нужно определить какие столбцы мы хотим загрузить из нашей таблицы. 

Так, для моего примера мне нужны столбцы Volume(unit)(2), SurfaceArea (um2)(5) + количество строк.

Шаг1. Извлечение данных из xlsx в df, с которыми я смогу работать в python

glob - показывает пути к файлам с указанным расширением и сохраняет в переменную filepaths

создаем пустой датафрейм, куда будут загружаться считанные столбики таблиц

добавила sheet_lst_wanted, чтобы сразу загружать несколько книг из из файла? 

+Есть проблема: не извлеклись все страницы -> собьется нумерация. Подумать... = рещила убрать все страницы и использовать отдельныей файлы 

в цикле объединяем датафреймы из считанных файлов 

In [6]:
from IPython import get_ipython
get_ipython().magic('reset -sf') 

import pandas as pd
import numpy as np
import glob


filepaths = glob.glob('**/N1*stack1*.xlsx', recursive=True) #можно изменять номера в N1*stack1,чтобы извлечь другие файлы
df = pd.DataFrame()
for find_files in filepaths:
    df = df.join (pd.read_excel(find_files, usecols=[1,4]), how='outer',lsuffix=' ')


Нужно изменить названия столбцов,чтобы было очевидно их происхождение из другого файла. 

Для этого была написана функция columns_rename, где в цикле добавляется порядковый номер файла, из которого происходило извлечение


Заменим измененным названиями столбцов действующие названия в датафрейме.

In [7]:
columns_names = [j for j in df.columns.tolist()]

def columns_rename(columns_names):
    j=1
    for i in range (0, len(columns_names), 2):
        columns_names[i] = 'N1_stack1_cell' +  str(j) + columns_names[i] #можно изменять номера в N1*stack1,чтобы отметить 
        columns_names[i+1] = 'N1_stack1_cell' +  str(j) + columns_names[i+1]    # принадлежность к другой партии данных
        j+=1
    return(columns_names)
df.set_axis(columns_rename(columns_names), axis=1, inplace=True)

Посмотрим на значения основных критериев описательной статистики

In [8]:
df.describe()

,N1_stack1_cell1Volume (unit),N1_stack1_cell1SurfaceArea,N1_stack1_cell2Volume (unit),N1_stack1_cell2SurfaceArea
count,3818.000000,3818.000000,3902.000000,3902.000000
mean,0.424083,4.567595,0.379947,4.037284
std,0.270616,2.123704,0.243734,1.827245
min,0.010741,0.333797,0.010741,0.333797
25%,0.225564,2.992793,0.204081,2.729861
50%,0.372359,4.350735,0.325814,3.790924
75%,0.572860,5.906711,0.501252,5.145034
max,2.842817,17.403682,2.345145,15.863077


Надо придумать как объединить разные файлы exel. Как быть с тем, что они имеют одинаковое название? + одинаковое название втолбцов + надо как-то разделить по клеткам и по животным